# ASP Bundle Adjust Plotting
## Examples for BlackSky Easton Glacier test case (n=20)
David Shean  
12/24/22

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors
import contextily as cx

In [ ]:
topdir = '/Users/dshean/scr/BlackSky/EastonGlacier_20220918-20221012/non-ortho'

In [ ]:
cd $topdir

In [ ]:
#ba_prefix = 'ba_all/ba_all'
ba_prefix = 'ba_all/ba_all_tri_weight'

In [ ]:
map_crs = 'EPSG:32610'

## Plot camera positions

In [ ]:
def read_cameras(csv_fn):
    cam_cols=['input_cam_file','x','y','z','r11','r12','r13','r21','r22','r23','r31','r32','r33']
    cam_df = pd.read_csv(csv_fn, header=0, names=cam_cols, index_col='input_cam_file')
    global_id = cam_df.index.to_series().str.split('-', expand=True)[1].astype('int') - 100
    cam_df['global_id'] = global_id
    cam_gdf = gpd.GeoDataFrame(cam_df, geometry=gpd.points_from_xy(cam_df['x'], cam_df['y'], cam_df['z'], crs='EPSG:4978'))
    return cam_gdf

In [ ]:
cam_init_csv = ba_prefix+'-initial-cameras.csv'
cam_final_csv = ba_prefix+'-final-cameras.csv'

In [ ]:
cam_init_gdf = read_cameras(cam_init_csv)
cam_final_gdf = read_cameras(cam_final_csv)

In [ ]:
cam_delta = cam_init_gdf[['x','y','z']] - cam_final_gdf[['x','y','z']]
cam_final_gdf['diff_m'] = np.sqrt(np.square(cam_delta).sum(axis=1))

In [ ]:
#For some reason, this doesn't yield same results as above
cam_final_gdf['diff_m_2'] = cam_final_gdf.distance(cam_init_gdf)

In [ ]:
#cam_init_idx = cam_init['input_cam_file'].str.split('/', expand=True)

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='global_id', cmap='tab20', legend='True', legend_kwds={'label': "BlackSky Satellite ID"})
#cx.add_basemap(ax, crs=map_crs)

In [ ]:
plot_kw = {'markersize':5}
ax = cam_init_gdf.to_crs(map_crs).plot(color='r', label='Initial', **plot_kw)
cam_final_gdf.to_crs(map_crs).plot(ax=ax, color='b', label='Final', **plot_kw)
ax.legend()
#cx.add_basemap(ax, crs=map_crs)
#ax.set_aspect('equal')

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='diff_m', norm=matplotlib.colors.LogNorm(), legend='True', legend_kwds={'label': "Position Difference (m)"})
#cx.add_basemap(ax, crs=map_crs)

### Compute rotation delta magnitude

In [ ]:
from scipy.spatial.transform import Rotation as R

In [ ]:
R_init = R.from_matrix(cam_init_gdf[['r11','r12','r13','r21','r22','r23','r31','r32','r33']].values.reshape((cam_init_gdf.shape[0],3,3)))
R_final = R.from_matrix(cam_final_gdf[['r11','r12','r13','r21','r22','r23','r31','r32','r33']].values.reshape((cam_final_gdf.shape[0],3,3)))

In [ ]:
#R_init.as_euler('ZYX', degrees=True)

In [ ]:
#R_final.as_euler('ZYX', degrees=True)

In [ ]:
eul_diff = (R_init.as_euler('ZYX', degrees=True) - R_final.as_euler('ZYX', degrees=True))

#### Testing Rotation Distance
http://www.boris-belousov.net/2016/12/01/quat-dist/ 

In [ ]:
R_final.as_matrix()[1]

In [ ]:
R_final.as_matrix()[1].T

In [ ]:
np.transpose(R_final.as_matrix(), axes=(0,2,1))[1]

In [ ]:
R_diff = R_init.as_matrix() * np.transpose(R_final.as_matrix(), axes=(0,2,1))

In [ ]:
R_diff[1]

In [ ]:
np.trace(R_diff, axis1=1, axis2=2)

In [ ]:
np.radians((np.trace(R_diff, axis1=1, axis2=2) - 1)/2)

In [ ]:
np.degrees(np.arccos(np.radians((np.trace(R_diff, axis1=1, axis2=2) - 1)/2)))

In [ ]:
cam_final_gdf['diff_deg'] = np.sqrt(np.square(eul_diff).sum(axis=1))

In [ ]:
ax = cam_final_gdf.to_crs(map_crs).plot(column='diff_deg', legend='True', legend_kwds={'label': "Orientation Difference (deg)"})
#cx.add_basemap(ax, crs=map_crs)

In [ ]:
def diff_plot():
    f, axa = plt.subplots(1,2, figsize=(10,5), sharex=True, sharey=True)
    norm=matplotlib.colors.LogNorm()
    #plot_kw = {'norm':norm, 's':1, 'legend':True, 'legend_kwds':{'label': col}}
    cam_final_gdf.to_crs(map_crs).plot(ax=axa[0], column='diff_m', legend='True', legend_kwds={'label': "Position Difference (m)"})
    #cx.add_basemap(ax=axa[1], crs=map_crs, attribution_size=0)
    cam_final_gdf.to_crs(map_crs).plot(ax=axa[1], column='diff_deg', legend='True', legend_kwds={'label': "Orientation Difference (deg)"})
    #cx.add_basemap(ax=axa[0], crs=map_crs, attribution_size=0)
    axa[0].set_title(f'Position Difference (m)')
    axa[1].set_title(f'Orientation Difference (deg)')
    plt.tight_layout()

In [ ]:
diff_plot()

In [ ]:
cam_final_gdf.iloc[0]

## Residuals

In [ ]:
def read_residuals(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mean_residual', 'num_observations']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    resid_df['from_DEM'] = resid_df['num_observations'].str.contains('# from DEM')
    resid_df['num_observations'] = resid_df['num_observations'].str.split('#', expand=True)[0].astype(int)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
resid_init_csv = ba_prefix+'-initial_residuals_pointmap.csv'
resid_final_csv = ba_prefix+'-final_residuals_pointmap.csv'

In [ ]:
resid_init = read_residuals(resid_init_csv)
resid_final = read_residuals(resid_final_csv)

In [ ]:
resid_init.describe()

In [ ]:
resid_final.describe()

In [ ]:
def resid_plot(resid_init, resid_final, col='mean_residual', clip_final=True, lognorm=False):
    f, axa = plt.subplots(1,2, figsize=(10,5), sharex=True, sharey=True)
    vmin = min(resid_init[col].min(), resid_final[col].min())
    vmax = max(resid_init[col].max(), resid_final[col].max())
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    if lognorm:
        norm = matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax)
    plot_kw = {'norm':norm, 's':1, 'legend':True, 'legend_kwds':{'label': col}}
    resid_final.sort_values(by=col).to_crs(map_crs).plot(ax=axa[1], column=col, **plot_kw)
    #cx.add_basemap(ax=axa[1], crs=map_crs, attribution_size=0)
    if clip_final:
        axa[0].autoscale(False)
    resid_init.sort_values(by=col).to_crs(map_crs).plot(ax=axa[0], column=col, **plot_kw)
    #cx.add_basemap(ax=axa[0], crs=map_crs, attribution_size=0)
    axa[0].set_title(f'Initial Residuals (n={resid_init.shape[0]})')
    axa[1].set_title(f'Final Residuals (n={resid_final.shape[0]})')
    plt.tight_layout()

In [ ]:
resid_plot(resid_init, resid_final, col='mean_residual', lognorm=True)

In [ ]:
resid_plot(resid_init, resid_final, col='num_observations')

## Consider points used during `--heights-from-DEM`

In [ ]:
idx = resid_init['from_DEM']

In [ ]:
resid_plot(resid_init[idx], resid_final[idx], col='mean_residual', lognorm=True)

In [ ]:
resid_plot(resid_init[~idx], resid_final[~idx], col='mean_residual', lognorm=True)

## Mapproject Residuals

In [ ]:
def read_mapproj_match_offset(csv_fn):
    resid_cols=['lon', 'lat', 'height_above_datum', 'mapproj_ip_dist_meters']
    resid_df = pd.read_csv(csv_fn, skiprows=2, names=resid_cols)
    resid_gdf = gpd.GeoDataFrame(resid_df, geometry=gpd.points_from_xy(resid_df['lon'], resid_df['lat'], crs='EPSG:4326'))
    return resid_gdf

In [ ]:
mapproj_match_offset_txt = ba_prefix+'-mapproj_match_offsets.txt'

In [ ]:
mapproj_match_offset = read_mapproj_match_offset(mapproj_match_offset_txt)

In [ ]:
mapproj_match_offset.describe()

In [ ]:
col='mapproj_ip_dist_meters'

In [ ]:
mapproj_match_offset.sort_values(by=col).to_crs(map_crs).plot(column=col, norm=matplotlib.colors.LogNorm(), legend=True)

In [ ]:
mapproj_match_offset.sort_values(by=col, ascending=False).to_crs(map_crs).plot(column=col, norm=matplotlib.colors.LogNorm(), legend=True)

## Geoplot tests for KDE

In [ ]:
import geoplot as gplt
import geoplot.crs as gcrs

In [ ]:
ax = gplt.pointplot(mapproj_match_offset, projection=gcrs.AlbersEqualArea(), s=1)
gplt.kdeplot(mapproj_match_offset[['mapproj_ip_dist_meters','geometry']], projection=gcrs.AlbersEqualArea(), ax=ax)

In [ ]:
gplt.kdeplot?

## Convergence angles

In [ ]:
conv_txt = ba_prefix+'-convergence_angles.txt'

In [ ]:
conv_cols = ['img1','img2','conv_25','conv_50','conv_75','num_angles']
conv = pd.read_csv(conv_txt, delimiter=' ', skiprows=1, header=0, names=conv_cols, index_col=False)
conv_valid = conv[conv['num_angles'] != 0]

In [ ]:
conv_valid.reset_index().plot.scatter(x='index', y='conv_50', c='num_angles', cmap='inferno')

In [ ]:
f, ax = plt.subplots()
m = ax.scatter(conv_valid.index, conv_valid['conv_50'], c=conv_valid['num_angles'], norm=matplotlib.colors.LogNorm())
plt.colorbar(m)